# Open Source Implementation of Snapchat Photo Filters with Python and Opencv

## Introduction

Our goal is to buid an application that enables users to apply filters to their images. We intend to create this application with open source tools such as Python and OpenCV. Specifically, our application will place a mask on the user's face in real-time. 

OpenCV (Open Source Computer Vision) is a library of programming functions mainly aimed at real-time computer vision. Originally developed by Intel's research center in Nizhny Novgorod (Russia), it was later supported by Willow Garage and is now maintained by Itseez.

https://en.wikipedia.org/wiki/OpenCV

## Snapchat Filters

Snapchat enables users to apply masks to their faces in real time, and to save snapshots of their masks to share with the world. 

![Image of Ahnold](http://forum.nutrimuscle.com/hebergeur_images/Upload/images/snapchat.jpg)


We did not investigate specifically how Snapchat does this; we only investigated doing this ourselves. Nevertheless, it appears that the application will require solving several different problems:

- Using the webcam to capture images in real-time
- Facial recognition
- Facial landmark recognition (for mask placement)
- Mask placement 

## Using the webcam with OpenCV

The code below works well to capture video from the webcam. The code is directly from the OpenCV documentation. 

In [ ]:
## http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html
'''
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
''';

## Facial Recognition

OpenCV exposes an API that allows users to train an object recognition model using Haar feature-based classifiers using the techniques in "Rapid Object Detection using a Boosted Cascade of Simple Features" (Viola, Jones, 2001). We use use it to construct our model for facial recognition. 

#### Haar Classifiers

Haar feature-based classification uses Adaboost to distinguish between important and irrelevant features in images. It also uses a Cascade of Classifiers method to quickly discard pieces of images where it detects no important features. 

http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html

#### Positive Samples

We need samples faces to train a model for facial recognition. They are used by the boosting process to define what the model should actually look for when trying to find your objects of interest. 

http://docs.opencv.org/trunk/dc/d88/tutorial_traincascade.html

We downloaded a dataset of over 13,000 faces from Labeled Faces in the Wild, a database of face photographs designed for studying the problem of unconstrained face recognition. The data set contains more than 13,000 images of faces collected from the web. Each face has been labeled with the name of the person pictured. 1680 of the people pictured have two or more distinct photos in the data set.:

http://vis-www.cs.umass.edu/lfw/lfw.tgz

![sample](Desktop/cv/faces/Aaron_Eckhart/Aaron_Eckhart_0001.jpg)


Each of the positive images have dimensions 250x250.

#### Negative Samples

We also need the negative samples as background noise and as examples of what *not* to recognize in an image. We downloaded a set of about 4000 negative samples from:

https://github.com/sonots/tutorial-haartraining/tree/master/data/negatives

Each of the photos is a basic background image that looks like this: 

![sample](Desktop/cv/negative_images/neg-0002.jpg)

Each of the negative images have dimensions 640x480.

#### Choosing 150 Random Faces for Annotation

We attempt to train a model with 150 faces chosen randomly from the 13,000 face database. The following code chooses 150 random faces:

In [1]:
## fix this

In [ ]:
import os, random, shutil
file_list = []
for i in range(1,151):
    random_file = random.choice(os.listdir('/cv/positive_images/'))
    source = '/cv/positive_images/' + random_file
    new_path = '/cv/train_faces/' + random_file
    os.rename(source, new_path)

#### Choosing 1000 Random Negatives for Training

In [ ]:
import os, random, shutil
file_list = []
for i in range(1,1001):
    random_file = random.choice(os.listdir('/Users/jaguirre/Desktop/cv/negative_images/'))
    source = '/Users/jaguirre/Desktop/cv/negative_images/' + random_file
    new_path = '/Users/jaguirre/Desktop/cv/train_negatives/' + random_file
    os.rename(source, new_path)

To point OpenCV to the list of negative files, we can easily create a list like this:

ls *.jpg > negatives.txt

#### OpenCV Functions for Model Creation

As mentioned above, the OpenCV API offers several utilities to facilitate model learning. We'll be using 3 of them:
- opencv_annotation
-- This function allows you identify objects in images and creates a text file of the location of the object within the image
- opencv_createsamples
-- This function creates samples by which to train your model
- opencv_traincascade
-- This function trains the model and uses several parameters for training


#### OpenCV Annotation

We ran the following code to annotate the faces in the 150 randomly selected files.

opencv_annotation -images ~/Desktop/cv/test_faces/ -annotations ~/Desktop/cv/annotations.txt

Running this command opens up each image in the directory for you to manually identify the target object.

![object recognition](http://christopher5106.github.io/img/rectangle_format.png)

#### OpenCV CreateSamples

We then create a positive sample set where the face images are superimposed onto the negative samples with the following command:

opencv_createsamples -num 149 -vec test_vec.vec -info annotations.txt -bg negatives.txt -w 250 -h 250

We got parse errors during this step; it's important to check the annotations.txt file to make sure that the number of points corresponds to the number of boxes in the images. If they do not correspond, they need to be edited manually. For example, if an image has 3 bounded boxes for 3 target objects, there should be 12 points associated with the image. 

#### OpenCV TrainCascade

We can now train a model to detect our target object with the following command:

opencv_traincascade -data ~/Desktop/cv/cascades -vec test_vec.vec -bg negatives.txt -featureType LBP -numPos 126 -acceptanceRatioBreakValue .00001

## Testing the Model

We can test our model with the following:

In [ ]:
import cv2

#cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier("cascades/cascade.xml")

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=4,
        minSize=(22, 22),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
## test on a picture

import numpy as np
import cv2 

face_cascade = cv2.CascadeClassifier('/Users/jaguirre/Desktop/cv/cascades/cascade.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 
img = cv2.imread('/Users/jaguirre/Desktop/cv/train_faces/Brenda_Magana_0001.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    #eyes = eye_cascade.detectMultiScale(roi_gray)
    #for (ex,ey,ew,eh) in eyes:
    #    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2) 

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()